<a href="https://colab.research.google.com/github/yohannesSM/datasets/blob/master/COSMIC_FPclassification_Bi_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import csv
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, LSTM, Dropout, Activation, Embedding, Bidirectional

In [3]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
vocab_size = 5000 # make the top list of words (common words)
embedding_dim = 64
max_length = 200
trunc_type = 'post'
padding_type = 'post'
oov_tok = '<OOV>' # OOV = Out of Vocabulary
training_portion = .8

In [12]:
#path = 'F:/anaconda/COSMIC_training_dataset.csv'
print(path)

F:/anaconda/COSMIC_training_dataset.csv


In [51]:
import pandas as pd

In [53]:
data = pd.read_csv('/content/COSMIC_training_dataset.csv')
data

,proj_id,FUR/US_id,FUR/US_desc,CFP,category
0,51,FUR_1,User opens the application to see on home scre...,6,medium
1,51,FUR_2,User clicks on the icon button ‘new’ to insert...,3,small
2,51,FUR_3,User selects an exam from the list in the home...,3,small
3,51,FUR_4,User clicks on the button ‘delete all’ to dele...,3,small
4,51,FUR_5,User selects an exam from the list in the home...,6,medium
...,...,...,...,...,...
20366,21g27-culrepo,1545,"As a researcher, I want to search within a rep...",5,medium
20367,21g27-culrepo,1546,"As an academicresearcher, I want to find the w...",5,medium
20368,21g27-culrepo,1547,"As an academicresearcher, I want to the thesis...",5,medium
20369,21g27-culrepo,1548,"As a patron, I want to search for report/datas...",5,medium


In [56]:
articles = []
labels = []

with open('COSMIC_training_dataset.csv','r') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    for row in reader:
        labels.append(row[4])
       # print(labels)
        article = row[2]
        for word in STOPWORDS:
            token = ' ' + word + ' '
            article = article.replace(token, ' ')
            article = article.replace(' ', ' ')
        articles.append(article)

['medium']
['medium', 'small']
['medium', 'small', 'small']
['medium', 'small', 'small', 'small']
['medium', 'small', 'small', 'small', 'medium']
['medium', 'small', 'small', 'small', 'medium', 'small']
['medium', 'small', 'small', 'small', 'medium', 'small', 'medium']
['medium', 'small', 'small', 'small', 'medium', 'small', 'medium', 'medium']
['medium', 'small', 'small', 'small', 'medium', 'small', 'medium', 'medium', 'medium']
['medium', 'small', 'small', 'small', 'medium', 'small', 'medium', 'medium', 'medium', 'small']
['medium', 'small', 'small', 'small', 'medium', 'small', 'medium', 'medium', 'medium', 'small', 'small']
['medium', 'small', 'small', 'small', 'medium', 'small', 'medium', 'medium', 'medium', 'small', 'small', 'small']
['medium', 'small', 'small', 'small', 'medium', 'small', 'medium', 'medium', 'medium', 'small', 'small', 'small', 'small']
['medium', 'small', 'small', 'small', 'medium', 'small', 'medium', 'medium', 'medium', 'small', 'small', 'small', 'small', 'smal

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [57]:
train_size = int(len(articles) * training_portion)

train_articles = articles[0: train_size]
train_labels = labels[0: train_size]

validation_articles = articles[train_size:]
validation_labels = labels[train_size:]

In [58]:
train_size

16296

In [59]:
print(train_articles)

['User opens application see home screen principal info included transcript, i.e., list exams, number exams, number credits average mark.', 'User clicks icon button ‘new’ insert data new exam database. The system provides error/confirmation messages.', 'User selects exam list home screen clicks button ‘delete’ delete database. The system provides error/confirmation messages.', 'User clicks button ‘delete all’ delete exams data database. The system provides error/confirmation messages.', 'User selects exam list home screen clicks button ‘update’ update data database. The system provides error/confirmation messages.', 'User selects exam list home screen clicks button ‘details’ see detailed info.', 'User clicks icon button ‘projection average’ system shows new box containing current average mark form. The user selects form number expected exams, number credits expected mark. The system provides expected average mark given input data values current exams.', 'User clicks button ‘export exam

In [64]:
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_articles)
word_index = tokenizer.word_index

In [63]:
tokenized_txt = tokenizer.fit_on_texts('The Disputes System must prevent external users from requesting original receipts. Requests for original receipts are restricted to internal users.')

In [65]:
train_sequences = tokenizer.texts_to_sequences(train_articles)

In [66]:
train_sequences = tokenizer.texts_to_sequences('The Disputes System must prevent external users from requesting original receipts. Requests for original receipts are restricted to internal users.')

In [67]:
print(train_sequences)

[[1327], [1], [216], [], [3194], [2], [339], [1], [4940], [1327], [216], [339], [], [339], [1], [339], [1327], [216], [1399], [], [1399], [4940], [339], [1327], [], [1], [1595], [216], [3451], [216], [2886], [1327], [], [216], [671], [1327], [216], [1595], [2886], [142], [3461], [], [4940], [339], [216], [1595], [339], [], [223], [1595], [1], [1399], [], [1595], [216], [1], [4940], [216], [339], [1327], [2], [2886], [823], [], [1], [1595], [2], [823], [2], [2886], [142], [3461], [], [1595], [216], [1400], [216], [2], [1], [1327], [339], [], [], [1595], [216], [1], [4940], [216], [339], [1327], [339], [], [223], [1], [1595], [], [1], [1595], [2], [823], [2], [2886], [142], [3461], [], [1595], [216], [1400], [216], [2], [1], [1327], [339], [], [142], [1595], [216], [], [1595], [216], [339], [1327], [1595], [2], [1400], [1327], [216], [3194], [], [1327], [1], [], [2], [2886], [1327], [216], [1595], [2886], [142], [3461], [], [4940], [339], [216], [1595], [339], []]


In [68]:
train_padded = pad_sequences(train_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)


In [69]:
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_articles)
word_index = tokenizer.word_index

train_sequences = tokenizer.texts_to_sequences(train_articles)
train_padded = pad_sequences(train_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

validation_sequences = tokenizer.texts_to_sequences(validation_articles)
validation_padded = pad_sequences(validation_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)


In [70]:
print(set(labels))

{'complex', 'medium', 'small', 'large'}


In [71]:
label_tokenizer = Tokenizer()
label_tokenizer.fit_on_texts(labels)

training_label_seq = np.array(label_tokenizer.texts_to_sequences(train_labels))
validation_label_seq = np.array(label_tokenizer.texts_to_sequences(validation_labels))

In [72]:
model  = Sequential()

In [73]:
model.add(Embedding(vocab_size, embedding_dim))
model.add(Dropout(0.5))
model.add(Bidirectional(LSTM(embedding_dim)))
model.add(Dense(6,activation='softmax'))

In [74]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 64)          320000    
                                                                 
 dropout (Dropout)           (None, None, 64)          0         
                                                                 
 bidirectional (Bidirectiona  (None, 128)              66048     
 l)                                                              
                                                                 
 dense (Dense)               (None, 6)                 774       
                                                                 
Total params: 386,822
Trainable params: 386,822
Non-trainable params: 0
_________________________________________________________________


In [75]:
opt = tf.keras.optimizers.Adam(lr=0.001, decay=1e-6)
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=opt,
    metrics=['accuracy'],
)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [76]:
num_epochs = 10
history = model.fit(train_padded, training_label_seq, epochs=num_epochs, validation_data=(validation_padded, validation_label_seq), verbose=2)


Epoch 1/10
510/510 - 127s - loss: 0.9164 - accuracy: 0.6974 - val_loss: 0.8016 - val_accuracy: 0.6979 - 127s/epoch - 250ms/step
Epoch 2/10
510/510 - 118s - loss: 0.7729 - accuracy: 0.7174 - val_loss: 0.5966 - val_accuracy: 0.7512 - 118s/epoch - 231ms/step
Epoch 3/10
510/510 - 118s - loss: 0.6236 - accuracy: 0.7703 - val_loss: 0.4255 - val_accuracy: 0.8378 - 118s/epoch - 231ms/step
Epoch 4/10
510/510 - 113s - loss: 0.5277 - accuracy: 0.8057 - val_loss: 0.3384 - val_accuracy: 0.8898 - 113s/epoch - 222ms/step
Epoch 5/10
510/510 - 115s - loss: 0.4650 - accuracy: 0.8286 - val_loss: 0.2662 - val_accuracy: 0.9104 - 115s/epoch - 226ms/step
Epoch 6/10
510/510 - 121s - loss: 0.4150 - accuracy: 0.8461 - val_loss: 0.2335 - val_accuracy: 0.9315 - 121s/epoch - 237ms/step
Epoch 7/10
510/510 - 120s - loss: 0.3737 - accuracy: 0.8620 - val_loss: 0.2038 - val_accuracy: 0.9308 - 120s/epoch - 236ms/step
Epoch 8/10
510/510 - 122s - loss: 0.3480 - accuracy: 0.8698 - val_loss: 0.1698 - val_accuracy: 0.9413 - 

In [79]:
req_txt = ["The Disputes System must prevent external users from requesting original receipts. Requests for original receipts are restricted to internal users"]

seq = tokenizer.texts_to_sequences(req_txt)
padded = pad_sequences(seq, maxlen=max_length)
pred = model.predict(padded)
labels = ['small', 'medium', 'large', 'complex'] 

print(pred)
print(np.argmax(pred))
print(labels[np.argmax(pred)-1])

[[1.7017008e-04 4.5711187e-01 2.0005718e-01 2.6551953e-01 7.7017210e-02
  1.2407234e-04]]
1
small


In [85]:
req_txt = ["The System shall be capable of integrating System Elements including Scientific Algorithms"]

seq = tokenizer.texts_to_sequences(req_txt)
padded = pad_sequences(seq, maxlen=max_length)
pred = model.predict(padded)
labels = ['complex', 'small', 'medium', 'large'] 

print(pred)
print(np.argmax(pred))
print(labels[np.argmax(pred)-1])

[[1.9902467e-04 4.4769388e-01 2.0202944e-01 2.7363569e-01 7.6295026e-02
  1.4695816e-04]]
1
complex


In [81]:
#saving our Bi-LSTM model
model.save('BiLSTM/COSMIC_FP_model.h5')

In [89]:
from keras import models  
from keras.models import load_model

In [90]:
#loading our LSTM model
model = load_model('BiLSTM/COSMIC_FP_model.h5')

In [ ]:
#we can predict now 